This tutorial is aimed at making Galaxian-inspired arcade game that we will call **Galaxy Invaders**. This tutorial draws some ideas from Chapter 5 of Roberto Ulloa's excellent _Kivy: Interactive Applications in Python_. Pick up a copy if you are looking for a nice introduction to making apps or games with Kivy! 

## 1.0 Debug Code


Before we even begin, let's introduce some debugging features that make it easier to lay out an app:

* First, put a rectangle around every widget we use.
* Next, add a `DebugLabel` class, a randomly-colored button spanning the parent object. Useful for figuring out what your layout looks like as you build it.

To make use of this code, add the following to your top-level `.kv` file. 
```
#:include debug.kv
```

When you are done, you can simply comment it out: 

```
##:include debug.kv
```

or remove it entirely.

Here's the complete `debug.kv`:


In [1]:
%%file debug.kv
#:import random random
    
<Widget>:
    canvas.after:
        Color:
            rgba: 1,1,1,.5
        Line:
            rectangle: self.x, self.y, self.width, self.height
            width: 2
                
<DebugLabel@Button>:
    size: self.parent.size
    pos: self.parent.pos
    background_color: random.random(), random.random(), random.random(), 0.6
    text: 'debuglabel'

Overwriting debug.kv


## 2.0 Basic Screen Layout
Let's divide the screen into areas.
The bottom 30% will be the **player area**. The top 70% will be the **enemy area**. To help visualize what we are laying out, we will add some `DebugLabels` as we go:

 

In [2]:
%%file galaxyinvaders.kv
#:include debug.kv
<GalaxyInvaders>:
    id: _mainscreen
    enemy_area: _enemy_area
    player_area: _player_area
    BoxLayout:
        orientation: 'vertical'
        FloatLayout:
            id: _enemy_area
            size_hint: 1, 0.7
            DebugLabel:
                text: 'Enemy Area'
        FloatLayout:
            id: _player_area
            size_hint: 1, 0.3
            DebugLabel:
                text: 'Player Area'


Overwriting galaxyinvaders.kv


### 2.1 IDs and References
There is a magic property in a `.kv` file called **id**. This assigns a label to the python object that will be created for the specified indentation block. By convention, we will prefix our id labels with an underscore; e.g.

 `id: _mainscreen`
 
Note this is only a convention. We don't *have* to do this, but sice it helps make our `.kv` file a little clearer, we will pretty much always do so. 

When this `.kv` file is loaded, it will add several properties to our `GalaxyInvaders` class: `enemy_area`, and `player_area`. These will contain refernces to the two FloatLayout objects we create on lines **9** and **14** of the `.kv` file. 

### 2.2 The Main Program
Now, let's add the basic python code we need to create and display this layout. As usual, we call our main program `main.py`.


In [3]:
%%file main.py
from kivy.app import App
from kivy.uix.floatlayout import FloatLayout

class GalaxyInvaders(FloatLayout):
    pass

class GalaxyInvadersApp(App):
    def build(self):
        return GalaxyInvaders()

if __name__ == '__main__':
    GalaxyInvadersApp().run()

Overwriting main.py


Finally, we can see what this looks like by doing a 
```
python main.py
```

In [4]:
%%html
<img src="assets/screenshots/galaxyinvaders-1.png" width=400 />

In [5]:
#!python main.py

### 2.3 Adding the Fleet Area

Now we need to add a placeholder for the **fleet**. A *fleet* is collection of placeholders (**docks**) for the invaders. In kivy terms, it will be a `GridLayout`. For now, you can think of the fleet is a moving rectangle inside the enemy area.

Because we will want to add methods to it later, we will create an object called `Fleet`, which is a kind of `GridLayout`.

In [6]:
%%file galaxyinvaders.kv
#:include debug.kv
<GalaxyInvaders>:
    id: _mainscreen
    enemy_area: _enemy_area
    player_area: _player_area
    BoxLayout:
        orientation: 'vertical'
        FloatLayout:
            id: _enemy_area
            size_hint: 1, 0.7
            DebugLabel:
                text: 'enemy area'
            Fleet:
                id: _fleet
                size_hint: .5, .4
                pos_hint: {'top': .9, 'center_x': 0.5}
                DebugLabel:
                    text: 'fleet'                    
        FloatLayout:
            id: _player_area
            size_hint: 1, 0.3
            DebugLabel:
                text: 'player area'


Overwriting galaxyinvaders.kv


Take notice of the `pos_hint` on line **17**. This will eventually burn us. We have used `center_x` to place the fleet in the middle of the screen, but we will eventually want to move the fleet by setting its `x` property directly. The guiding principle of using Layouts in kivy is that if you want absolute (pixel-level) control, vs relative (`size_hint` and `pos_hint`), you have to set the appropriate hint to `None`.

We will leave it for now. But note eventually we will need to set it to something like
```
pos_hint: {'top': 0.9}
x: Window.width/2 - Window.width/4
```


Now we declare `Fleet` to be a `GridLayout` in `main.py`.

In [7]:
%%file main.py
from kivy.app import App
from kivy.uix.floatlayout import FloatLayout
from kivy.uix.gridlayout import GridLayout

class Fleet(GridLayout):
    pass

class GalaxyInvaders(FloatLayout):
    pass

class GalaxyInvadersApp(App):
    def build(self):
        return GalaxyInvaders()

if __name__ == '__main__':
    GalaxyInvadersApp().run()

Overwriting main.py


In [8]:
%%html
<img src="assets/screenshots/galaxyinvaders-2.png" width=400 />

In [9]:
#!python main.py

## 3.0 Sprites: Images and Atlases
Let's create the basic sprites for our game. We will need
* The player's ship (`Player`), 
* An alien ship (`Invader`)
* An explosion (`Boom`)
* Player and Invader `Projectile` (`Missile` and `Bomb` respectively)

However you acquire these images, for efficiency reasons, you should put them inside an **atlas** (also known as a *spritesheet*. We can generate an atlas/spritesheet from a set of files using kivy: `python -m kivy.atlas basename size filelist`; e.g.

 `python -m kivy.atlas galaxy 100 *.png`

Here _size_ is the maximum dimension of the spritesheet.

Doing the above, you get an atlas file that looks like this:



In [10]:
%%file assets/sprites/galaxy.atlas
{"galaxy-0.png": {"missile": [42, 35, 10, 13], 
                  "player": [52, 50, 48, 48], 
                  "invader": [2, 50, 48, 48], 
                  "boom": [2, 18, 26, 30], 
                  "bomb": [30, 29, 10, 19]}}

Overwriting assets/sprites/galaxy.atlas


Now we can create image objects from these sprites. (Note, you should obtain the sizes from the generated atlas file, or the original sprites):

In [139]:
%%file sprites.kv
<Invader>:
    source: 'atlas://assets/sprites/galaxy/invader'
    size_hint: None, None
    size: 40, 40
<Player>:
    source: 'atlas://assets/sprites/galaxy/player'
    size_hint: None, None
    size: 40, 40
<Boom>:
    source: 'atlas://assets/sprites/galaxy/boom'
    size_hint: None, None
    size: 26, 30
<Missile>:
    source: 'atlas://assets/sprites/galaxy/missile'
    size_hint: None, None
    size: 12, 15
<Bomb>:
    source: 'atlas://assets/sprites/galaxy/bomb'
    size_hint: None, None
    size: 12, 27
            

Overwriting sprites.kv


Let's create a new class for the **Player**

In [140]:
%%file player.py
from kivy.uix.image import Image

class Player(Image):
    pass

Overwriting player.py


Let's load the `sprites.kv` kivy file manually. We will include our new player object, and then call `kivy.lang.builder.load_file()` to read the associated `.kv` file.

In [141]:
%%file main.py
from kivy.app import App
from kivy.uix.floatlayout import FloatLayout
from kivy.uix.gridlayout import GridLayout
from kivy.lang.builder import Builder
from player import Player

Builder.load_file('sprites.kv')

class Fleet(GridLayout):
    pass

class GalaxyInvaders(FloatLayout):
    pass

class GalaxyInvadersApp(App):
    def build(self):
        return GalaxyInvaders()

if __name__ == '__main__':
    GalaxyInvadersApp().run()

Overwriting main.py


The only change necessary to the `galaxyinvaders.kv` is to add the `Player` object to the `player_area`. We are not going to use a `pos_hint` to position it, as we want to be able to move it via x coordinate later, so we will use an absolute x coordinate. 

In [142]:
%%file galaxyinvaders.kv
#:include debug.kv
<GalaxyInvaders>:
    id: _mainscreen
    enemy_area: _enemy_area
    player_area: _player_area
    player: _player
    fleet: _fleet
    BoxLayout:
        orientation: 'vertical'
        FloatLayout:
            id: _enemy_area
            size_hint: 1, 0.7
            DebugLabel:
                text: 'enemy area'
            Fleet:
                id: _fleet
                size_hint: .5, .4
                pos_hint: {'top': .9, 'center_x': 0.5}
                DebugLabel:
                    text: 'fleet'                    
        FloatLayout:
            id: _player_area
            size_hint: 1, 0.3
            Player:
                id: _player
                mainscreen: _mainscreen
                player_area: _player_area
                x: self.parent.width / 2


Overwriting galaxyinvaders.kv


Since we have starting using the player area, we can safely dropped the `DebugLabel` code that used to identify it

In [143]:
%%html
<img src="assets/screenshots/galaxyinvaders-3.png" width=400 />

In [144]:
#!python main.py

## 4.0 Controls
Let's make the player move. We can use touches for this, the keyboard, or both. We are obviously going to do both.

### 4.1 Using the Keyboard
First, let's bind movement to the **left** and **right** arrow keys.

In our app initialization, we will get a handle to a keyboard (if present) using the Window object's  `request_keyboard()` method. Then we can set up a handler for keypresses. Since we are mapping keys, we may as well use **Esc** to quit our game. 

Notice we use `App.get_running_app()` to get a handle to our `GalaxyInvadersApp` instance so that we can shut it down. This technique for accessing the root app is going to be very handy when we get around to adding sound.

In [145]:
%%file main.py
from kivy.app import App
from kivy.uix.floatlayout import FloatLayout
from kivy.uix.gridlayout import GridLayout
from kivy.lang.builder import Builder
from kivy.core.window import Window
from kivy.logger import Logger

from player import Player

Builder.load_file('sprites.kv')

class Fleet(GridLayout):
    pass

class GalaxyInvaders(FloatLayout):
    def __init__(self, **kwargs):
        super(GalaxyInvaders, self).__init__(**kwargs)
        self._keyboard = Window.request_keyboard(self.close, self)
        self._keyboard.bind(on_key_down=self.press)
        
    def close(self):
        self._keyboard.unbind(on_key_down=self.press)
        self._keyboard = None
        # Eventually, we should do an 'Are You Sure?' prompt. For now, just quit
        App.get_running_app().stop() 
        
    def press(self, keyboard, keycode, text, modifiers):
        if keycode[1] == 'left':
            self.player.x -= 30
            if self.player.x < self.x:
                self.player.x = self.x
        elif keycode[1] == 'right':
            self.player.x += 30
            if self.player.x > self.width - self.player.width:
                self.player.x = self.width - self.player.width
        elif keycode[1] == 'escape':
            self.close()
        else:
            Logger.debug("Unknown key: {}".format(keycode))
        return True

class GalaxyInvadersApp(App):
    def build(self):
        return GalaxyInvaders()

if __name__ == '__main__':
    GalaxyInvadersApp().run()

Overwriting main.py


Also notice (Lines **7**, **40**) that we are using the kivy `Logger` class to write out debug information.

In [146]:
%%html
<img src="assets/screenshots/galaxyinvaders-4a.png" width=400 />

In [147]:
#!python main.py

### 4.2 Using Touches

Keyboard is great and all, but touch screens are half the fun of kivy, so let's add those. 

So, how should we move our object by touch? Our first idea is simply by dragging it. We will start a move by touching **on** our object. Since we can drag off of our object (but we want the ship to continue moving), we will want to **grab** the touch to ensure we get all later messages. 


In [148]:
%%file player.py
from kivy.uix.image import Image
from kivy.logger import Logger

class Player(Image):
    def on_touch_down(self, touch):
        if self.collide_point(*touch.pos):
            self.center_x = touch.x
            touch.grab(self)
            return True
            
    def on_touch_move(self, touch):
        if touch.grab_current is self:
            self.center_x = touch.x
            return True


Overwriting player.py


Kivy has a funny (but useful) design for messages. All widgets receive all messages, regardless of where the originate. So in this case, every widget will receive an `on_touch_down` event. It's up to the individual widgets to check if this message is destined for them.

In our code, we perform this test by checking `if self.collide_point(*touch_pos)` (Line **7**); i.e. if the touch collides with the widget `self`.

Similarly, all widgets will receive the `on_touch_move` message. Since we only want to move the ship if the drag started with a touch on the `Player` object, we will use the grab information to do this test for us (Line **13**).

Try it out!

In [149]:
%%html
<img src="assets/screenshots/galaxyinvaders-4b.png" width=400 />

In [150]:
#!python main.py

## 5.0 Projectiles: Bombs and Missiles
There will be 2 kinds of projectiles in the game:
* **Missiles**, fired by the player
* **Bombs**, fired by the invaders

We will use a single class (`Projectile`) for both types, and override as necessary.

In [151]:
%%file projectile.py
from kivy.animation import Animation
from kivy.uix.image import Image

class Projectile(Image):
    '''Lauch this piece of ammunition towards a target object (`target`),
    located at coordinates (`tx`, `ty`).
    
    When the projectile reaches its target, it disappears. Collision handling is done elsewhere.

    The `target` object is used to determine which collide_projectile method to check for collision
    in the on_progress method of the projectile.
    '''
    
    def shoot(self, tx, ty, target):
        self.target = target
        self.animation = Animation(x=tx, y=ty)
        self.animation.bind(on_start=self.on_start)
        self.animation.bind(on_progress=self.on_progress)
        self.animation.bind(on_complete=self.on_stop)
        self.animation.start(self)
        
    def on_start(self, instance, value):
        pass
    
    def on_progress(self, instance, value, progression):
        pass
            
    def on_stop(self, instance, value):
        self.parent.remove_widget(self)
        

class Missile(Projectile):
    pass


class Bomb(Projectile):
    pass

Overwriting projectile.py


We already saw how to bind events to keypresses, so let's bind "shoot" to **Spacebar**.


In [152]:
%%file main.py
from kivy.app import App
from kivy.uix.floatlayout import FloatLayout
from kivy.uix.gridlayout import GridLayout
from kivy.lang.builder import Builder
from kivy.core.window import Window
from kivy.logger import Logger

from player import Player

Builder.load_file('sprites.kv')

class Fleet(GridLayout):
    pass

class GalaxyInvaders(FloatLayout):
    def __init__(self, **kwargs):
        super(GalaxyInvaders, self).__init__(**kwargs)
        self._keyboard = Window.request_keyboard(self.close, self)
        self._keyboard.bind(on_key_down=self.press)
        
    def close(self):
        self._keyboard.unbind(on_key_down=self.press)
        self._keyboard = None
        # Eventually, we should do an 'Are You Sure?' prompt. For now, just quit
        App.get_running_app().stop() 
        
    def press(self, keyboard, keycode, text, modifiers):
        if keycode[1] == 'left':
            self.player.x -= 30
            if self.player.x < self.x:
                self.player.x = self.x
        elif keycode[1] == 'right':
            self.player.x += 30
            if self.player.x > self.width - self.player.width:
                self.player.x = self.width - self.player.width
        elif keycode[1] == 'escape':
            self.close()
        elif keycode[1] == 'spacebar':
            self.player.shoot()
        else:
            Logger.debug("Unknown key: {} {}".format(keycode, modifiers))
        return True

class GalaxyInvadersApp(App):
    def build(self):
        return GalaxyInvaders()

if __name__ == '__main__':
    GalaxyInvadersApp().run()

Overwriting main.py


Lines **39-40** handle the keyboard shooing. As for the touch controls, any touch in the player area (that isn't directly on the player) will shoot. This is handled in lines **13-15** below.

In [153]:
%%file player.py
from kivy.uix.image import Image
from kivy.logger import Logger
from kivy.graphics import Line, Ellipse
from projectile import Missile

class Player(Image):
    def on_touch_down(self, touch):
        if self.collide_point(*touch.pos):
            self.center_x = touch.x
            touch.grab(self)
            return True
        elif self.player_area.collide_point(*touch.pos):
            self.shoot()
            return True
            
    def on_touch_move(self, touch):
        if touch.grab_current is self:
            self.center_x = touch.x
            return True
        
    def shoot(self):
        '''Shoot straight up.'''
        missile = Missile()
        missile.center = (self.center_x, self.top)
        self.mainscreen.add_widget(missile)
        (fx, fy) = self.center_x, self.mainscreen.height
        missile.shoot(fx, fy, self.mainscreen.fleet)
        

Overwriting player.py


Finally the shot method (starting on line **22**) handles sending the missile on its way

In [154]:
%%html
<img src="assets/screenshots/galaxyinvaders-5b.png" width=400 />

In [155]:
#!python main.py

## 6.0 Explosions and Sound Effects
Let's add an explosion when the missile hits something. Since we haven't added enemies yet, the only thing we have to hit is the top of the screen. We can do this by adding an explosion sprite in the projectile's `on_stop` handler

In [156]:
%%file projectile.py
from kivy.animation import Animation
from kivy.uix.image import Image
from kivy.logger import Logger
from boom import Boom

class Projectile(Image):
    '''Lauch this piece of ammunition towards a target object (`target`),
    located at coordinates (`tx`, `ty`).
    
    When the projectile reaches its target, it disappears. Collision handling is done elsewhere.

    The `target` object is used to determine which collide_projectile method to check for collision
    in the on_progress method of the projectile.
    '''
    
    def shoot(self, tx, ty, target):
        self.target = target
        self.animation = Animation(x=tx, y=ty-40)
        self.animation.bind(on_start=self.on_start)
        self.animation.bind(on_progress=self.on_progress)
        self.animation.bind(on_complete=self.on_stop)
        self.animation.start(self)
        
    def on_start(self, instance, value):
        pass
    
    def on_progress(self, instance, value, progression):
        pass
            
    def on_stop(self, instance, value):
        boom = Boom()
        boom.pos = self.pos
        self.parent.add_widget(boom)
        self.parent.remove_widget(self)

        
class Missile(Projectile):
    pass


class Bomb(Projectile):
    pass

Overwriting projectile.py


Note, we have added a gross hack on line **19**, to stop the projectile 30 pixels short of its target. This gives us enough space to draw the explosion. We will fix this hack soon.

In [157]:
#!python main.py

There are obviously two things wrong with this explosion.
* It sticks around forever: it should eventually disappear, and
* it doesn't go "boom". 

For the former, let's schedule a removal using the `Clock`.


In [158]:
%%file boom.py
from kivy.uix.image import Image
from kivy.clock import Clock


class Boom(Image):
    def __init__(self, **kwargs):
        super(Boom, self).__init__(**kwargs)
        Clock.schedule_once(lambda dt:self.parent.remove_widget(self), 0.2)


Overwriting boom.py


The `lambda` function on line **9** fixes the fact that `schedule_once` wants a function that takes a parameter `dt`. The `lambda` function creates an unnamed function that ignores this argument.


In [159]:
#!python main.py

For the second problem, we need to add sound. Let's get to it!

### 6.1 The wrong way to do sound
It seems like we could just play the sound as part of the `boom` constructor, but this runs into a subtle problem: the sound can only be played once. Try firing multiple bombs and listening for the explosions.



In [160]:
%%file boom.py
from kivy.uix.image import Image
from kivy.clock import Clock
from kivy.core.audio import SoundLoader


class Boom(Image):
    sound = SoundLoader.load('assets/sound/boom.wav')
    def __init__(self, **kwargs):
        super(Boom, self).__init__(**kwargs)
        self.play_sound()
        Clock.schedule_once(lambda dt:self.parent.remove_widget(self), 0.2)

    def play_sound(self):
        self.sound.play()
        

Overwriting boom.py


In [161]:
#!python main.py

You'll hear one explosion, and when it finishes, you'll hear the next. This isn't what we want. We want sounds that overlap. To do this, we need to do something a little more drastic.

### 6.2 The right way to do sound
Here we will create a `Sounds` object with a magic `play_name` method that loads a sound called `name.wav`. Whenver a sound is played, it checks to see if there is a `SoundLoader` with that name that is currently **stopped**. If yes, it plays the sound. If no, it adds a new `SoundLoader` and plays the sound. 



In [162]:
%%file sounds.py
from kivy.core.audio import SoundLoader
from kivy.logger import Logger

class Sounds(object):

    def __init__(self, **kw):
        self.sounds = {}
        super(Sounds, self).__init__(**kw)
    
    def __getattr__(self, attr, volume=None):
        if not attr.startswith('play_'):
            return object.__getattribute__(self, attr)
        f = attr.split('play_')[1]
        sounds = getattr(self, 'sounds')
        loaded = sounds.get(f, [])
        ready = None
        for l in loaded:
            if l.state == 'stop':
                ready = l
                break
        if ready == None:
            ready = SoundLoader.load('assets/sound/' + f + '.wav')
            sounds[f] = loaded
            loaded.append(ready)
            Logger.debug("Sounds: Loading {}. {} loaded".format(f, len(sounds[f])))
        return ready.play
    

Overwriting sounds.py


In [163]:
%%file main.py
from kivy.app import App
from kivy.uix.floatlayout import FloatLayout
from kivy.uix.gridlayout import GridLayout
from kivy.lang.builder import Builder
from kivy.core.window import Window
from kivy.logger import Logger

from player import Player
from sounds import Sounds

Builder.load_file('sprites.kv')


class Fleet(GridLayout):
    pass


class GalaxyInvaders(FloatLayout):
    def __init__(self, **kwargs):
        super(GalaxyInvaders, self).__init__(**kwargs)
        self._keyboard = Window.request_keyboard(self.close, self)
        self._keyboard.bind(on_key_down=self.press)
        
    def close(self):
        self._keyboard.unbind(on_key_down=self.press)
        self._keyboard = None
        # Eventually, we should do an 'Are You Sure?' prompt. For now, just quit
        App.get_running_app().stop() 
        
    def press(self, keyboard, keycode, text, modifiers):
        if keycode[1] == 'left':
            self.player.x -= 30
            if self.player.x < self.x:
                self.player.x = self.x
        elif keycode[1] == 'right':
            self.player.x += 30
            if self.player.x > self.width - self.player.width:
                self.player.x = self.width - self.player.width
        elif keycode[1] == 'escape':
            self.close()
        elif keycode[1] == 'spacebar':
            self.player.shoot()
        else:
            Logger.debug("Unknown key: {}".format(keycode))
        return True

class GalaxyInvadersApp(App):
    sounds = Sounds()
    def build(self):
        return GalaxyInvaders()

if __name__ == '__main__':
    GalaxyInvadersApp().run()

Overwriting main.py


Now, `App.get_running_app().sounds` seems a bit awkward, but I haven't yet found a better place to put it.


In [164]:
%%file boom.py
from kivy.uix.image import Image
from kivy.core.audio import SoundLoader
from kivy.app import App
from kivy.clock import Clock


class Boom(Image):
    def __init__(self, **kwargs):
        super(Boom, self).__init__(**kwargs)
        self.play_sound()
        Clock.schedule_once(lambda dt:self.parent.remove_widget(self), 0.2)
        
    def play_sound(self):
        App.get_running_app().sounds.play_boom3()
        

Overwriting boom.py


In [165]:
#!python main.py

## 7.0 Building the Invasion Fleet

Most of the time, enemy invaders fly in formation. In Galaxian, and similar games, enemies sometimes leave this formation to go on a bombing run. We will ignore this behavior for now, and focus on the movement in formation. We call this formation the `Fleet`. The `Fleet` is really a collection of `Dock`s; i.e. places for invaders to sit while they move in formation. We will add these docks using a `GridLayout`.

In [166]:
%%file dock.py
from kivy.uix.widget import Widget

class Dock(Widget):
    pass

Overwriting dock.py


We need to add the right number of rows and columns to the fleet `GridLayout`, and change the pos_hint to not specify the x coordinate, as we will use it directly to do movement. In this case, we will specify 8 columns. We will finally lose our `DebugLabels` while we are at it:

In [167]:
%%file galaxyinvaders.kv
#:include debug.kv
<GalaxyInvaders>:
    id: _mainscreen
    enemy_area: _enemy_area
    player_area: _player_area
    player: _player
    fleet: _fleet
    BoxLayout:
        orientation: 'vertical'
        FloatLayout:
            id: _enemy_area
            size_hint: 1, 0.7
            Fleet:
                id: _fleet
                mainscreen: _mainscreen
                size_hint: .5, .4
                pos_hint: {'top': .9}
                x: root.width/2 - root.width/4
                cols: 8
                spacing: 20
                    
        FloatLayout:
            id: _player_area
            size_hint: 1, 0.3
            Player:
                id: _player
                mainscreen: _mainscreen
                player_area: _player_area
                x: self.parent.width / 2

Overwriting galaxyinvaders.kv


We initialize the docks in the `Fleet` constructor:

In [168]:
%%file fleet.py

from kivy.uix.gridlayout import GridLayout
from kivy.core.window import Window
from dock import Dock

class Fleet(GridLayout):
    
    def __init__(self, **kwargs):
        super(Fleet, self).__init__(**kwargs)
        for x in range (0,32):
            dock = Dock()
            self.add_widget(dock)
        self.center_x = Window.width/4
        

Overwriting fleet.py


We load the new  load the fleet on line **11** of `main.py`, replacing the old empty class declaration.

In [169]:
%%file main.py
from kivy.app import App
from kivy.uix.floatlayout import FloatLayout
from kivy.uix.gridlayout import GridLayout
from kivy.lang.builder import Builder
from kivy.core.window import Window
from kivy.logger import Logger
from kivy.core.audio import SoundLoader

from player import Player
from fleet import Fleet
from sounds import Sounds

Builder.load_file('sprites.kv')


class GalaxyInvaders(FloatLayout):
    def __init__(self, **kwargs):
        super(GalaxyInvaders, self).__init__(**kwargs)
        self._keyboard = Window.request_keyboard(self.close, self)
        self._keyboard.bind(on_key_down=self.press)
        
    def close(self):
        self._keyboard.unbind(on_key_down=self.press)
        self._keyboard = None
        # Eventually, we should do an 'Are You Sure?' prompt. For now, just quit
        App.get_running_app().stop() 
        
    def press(self, keyboard, keycode, text, modifiers):
        if keycode[1] == 'left':
            self.player.x -= 30
            if self.player.x < self.x:
                self.player.x = self.x
        elif keycode[1] == 'right':
            self.player.x += 30
            if self.player.x > self.width - self.player.width:
                self.player.x = self.width - self.player.width
        elif keycode[1] == 'escape':
            self.close()
        elif keycode[1] == 'spacebar':
            self.player.shoot()
        else:
            Logger.debug("Unknown key: {}".format(keycode))
        return True

class GalaxyInvadersApp(App):
    sounds = Sounds()
    def build(self):
        return GalaxyInvaders()

if __name__ == '__main__':
    GalaxyInvadersApp().run()

Overwriting main.py


In [170]:
%%html
<img src="assets/screenshots/galaxyinvaders-7.png" width=400 />

In [171]:
#!python main.py

### 7.1 Animating the Fleet
First we used animations to fire projectiles. Now we are going to use them to move the fleet.

We set a property, `move_delay` to indicate how long it takes for the fleet to move from side to side.

In [172]:
%%file fleet.py
from kivy.uix.gridlayout import GridLayout
from kivy.animation import Animation
from kivy.clock import Clock
from kivy.core.window import Window
from kivy.properties import NumericProperty
from kivy.logger import Logger

from random import randint, random
from dock import Dock

class Fleet(GridLayout):
    move_delay = NumericProperty(3)
    
    def __init__(self, **kwargs):
        super(Fleet, self).__init__(**kwargs)
        for x in range (0,32):
            dock = Dock()
            self.add_widget(dock)
    
    def start_fleet(self, instance=None, value=None):
        '''Start the fleet march'''
        self.x = Window.width/2 - Window.width/4 # why do I need this?
        self.go_left(instance, None)
        
    def go_left(self, instance, value):
        '''Move the fleet towards the left edge of the screen. 
        Normally the fleet moves from the right side of the screen.
        If if `value` is None, however, we are calling the animation for the first time,
        in which case, the animation starts from the middle of the screen, so we half the
        nimation duration'''
        if value is None:
            animation = Animation(x=0, d=self.move_delay / 2.0)
        else:
            animation = Animation(x=0, d=self.move_delay)
        animation.bind(on_complete = self.go_right)
        animation.start(self)
    
    def go_right(self, instance, value):
        '''Move the fleet towards the right edge of the screen.'''
        animation = Animation(right=self.parent.width, d=self.move_delay)
        animation.bind(on_complete=self.go_left)
        animation.start(self)
    


Overwriting fleet.py


Notice the way we loop animations. We schedule the next animation as part of the animation's `on_complete` event.

In [173]:
%%file main.py
from kivy.app import App
from kivy.uix.floatlayout import FloatLayout
from kivy.uix.gridlayout import GridLayout
from kivy.lang.builder import Builder
from kivy.core.window import Window
from kivy.logger import Logger
from kivy.core.audio import SoundLoader
from kivy.clock import Clock
from kivy.uix.label import Label
from kivy.animation import Animation

from player import Player
from sounds import Sounds
from fleet import Fleet

Builder.load_file('sprites.kv')


class GalaxyInvaders(FloatLayout):
    def __init__(self, **kwargs):
        super(GalaxyInvaders, self).__init__(**kwargs)
        self._keyboard = Window.request_keyboard(self.close, self)
        self._keyboard.bind(on_key_down=self.press)
        self.start_game()
        
    def start_game(self):
        self.fleet.start_fleet(self.fleet)

    def close(self):
        self._keyboard.unbind(on_key_down=self.press)
        self._keyboard = None
        # Eventually, we should do an 'Are You Sure?' prompt. For now, just quit
        App.get_running_app().stop() 
        
    def press(self, keyboard, keycode, text, modifiers):
        if keycode[1] == 'left':
            self.player.x -= 30
            if self.player.x < self.x:
                self.player.x = self.x
        elif keycode[1] == 'right':
            self.player.x += 30
            if self.player.x > self.width - self.player.width:
                self.player.x = self.width - self.player.width
        elif keycode[1] == 'escape':
            self.close()
        elif keycode[1] == 'spacebar':
            self.player.shoot()
        else:
            Logger.debug("Unknown key: {}".format(keycode))
        return True

class GalaxyInvadersApp(App):
    sounds = Sounds()
    def build(self):
        return GalaxyInvaders()

if __name__ == '__main__':
    GalaxyInvadersApp().run()

Overwriting main.py


In [174]:
%%html
<img src="assets/screenshots/galaxyinvaders-8.png" width=400 />

In [175]:
#!python main.py

### 7.2 A Diversion: Spit and Polish
Let's take a moment to add some polish. Let's explain the keys to the user, and add the 's' key to start. Also, prepare the user with an introductory animation.

In [176]:
%%file main.py
from kivy.app import App
from kivy.uix.floatlayout import FloatLayout
from kivy.uix.gridlayout import GridLayout
from kivy.lang.builder import Builder
from kivy.core.window import Window
from kivy.logger import Logger
from kivy.core.audio import SoundLoader
from kivy.clock import Clock
from kivy.uix.label import Label
from kivy.animation import Animation

from player import Player
from sounds import Sounds
from fleet import Fleet

Builder.load_file('sprites.kv')


class GalaxyInvaders(FloatLayout):
    playing = False
    unlocked = False
    message = None

    def __init__(self, **kwargs):
        super(GalaxyInvaders, self).__init__(**kwargs)
        self._keyboard = Window.request_keyboard(self.close, self)
        self._keyboard.bind(on_key_down=self.press)
        label = Label(text="Press [b]S[/b] to start\n[b]<- ->[/b] to move\n[b]Space[/b] to shoot\n[b]Esc[/b] to quit",
                     halign='center', markup=True)
        self.add_widget(label)
        self.help_text = label
        
    def start_game(self):
        '''Display a  'ready' message before a game starts.
        When the animation is done, start the game.'''
        label = Label(text='Ready!')
        animation = Animation(font_size=72, d=2)
        animation += Animation(font_size=0, d=2)
        self.add_widget(label)
        self.message = label
        animation.bind(on_complete=self.fleet.start_fleet)
        animation.bind(on_complete=self.remove_message)
        animation.start(label)
        
    def remove_message(self, instance, value):
        '''Remove the message text when the animation is done'''
        self.remove_widget(self.message)
        self.message = None

    def close(self):
        self._keyboard.unbind(on_key_down=self.press)
        self._keyboard = None
        # Eventually, we should do an 'Are You Sure?' prompt. For now, just quit
        App.get_running_app().stop() 
        
    def press(self, keyboard, keycode, text, modifiers):
        '''Handle key commands, with different modes for playing, not playing'''
        if keycode[1] == 'escape':
                self.close()
                return True

        if not self.playing:
            if keycode[1] == 's':
                self.remove_widget(self.help_text)
                self.playing = True
                self.start_game()
        else:
            if keycode[1] == 'left':
                self.player.x -= 30
                if self.player.x < self.x:
                    self.player.x = self.x
            elif keycode[1] == 'right':
                self.player.x += 30
                if self.player.x > self.width - self.player.width:
                    self.player.x = self.width - self.player.width
            elif keycode[1] == 'spacebar':
                self.player.shoot()
            else:
                Logger.debug("Unknown key: {}".format(keycode))
        return True

class GalaxyInvadersApp(App):
    sounds = Sounds()
    def build(self):
        return GalaxyInvaders()

if __name__ == '__main__':
    GalaxyInvadersApp().run()

Overwriting main.py


In [177]:
%%html
<img src="assets/screenshots/galaxyinvaders-9.png" width=400 />

In [178]:
#!python main.py

### 7.3 Adding the Invaders

Finally, let's put some invaders in there.

In [179]:
%%file invader.py

from kivy.uix.image import Image

class Invader(Image):
    pass

Overwriting invader.py


Invaders can be **docked** or **undocked**. If undocked, they will be following an attacking trajectory. Start with everything docked.

In [180]:
%%file dock.py
from kivy.uix.widget import Widget
from invader import Invader

class Dock(Widget):
    def __init__(self, **kwargs):
        super(Dock, self).__init__(**kwargs)
        self.invader = Invader()
        self.add_widget(self.invader)
        self.bind_invader()
        
    def bind_invader(self, instance=None, value=None):
        self.invader.formation = True
        self.bind(pos = self.on_pos)
        
    def unbind_invader(self):
        self.invader.formation = False
        self.unbind(pos = self.on_pos)
        
    def on_pos(self, instance, value):
        self.invader.pos = self.pos
            

Overwriting dock.py


Line **14** handles the magic task of updating the invader position whenever the dock moves. Binding a function to a kivy property means the function is called whenever that property changes. 

In [181]:
%%html
<img src="assets/screenshots/galaxyinvaders-10.png" width=400 />

In [182]:
#!python main.py

At random intervals, the invaders should drop a bomb. Add this to the fleet code.

In [183]:
%%file fleet.py
from kivy.uix.gridlayout import GridLayout
from kivy.animation import Animation
from kivy.clock import Clock
from kivy.core.window import Window
from kivy.properties import NumericProperty, ListProperty
from kivy.logger import Logger

from random import choice, random
from dock import Dock

class Fleet(GridLayout):
    survivors = ListProperty([])
    move_delay = NumericProperty(3)
    
    def __init__(self, **kwargs):
        super(Fleet, self).__init__(**kwargs)
        for x in range (0,32):
            dock = Dock()
            self.survivors.append(dock)
            self.add_widget(dock)
    
    def start_fleet(self, instance=None, value=None):
        '''Start the fleet march'''
        self.x = Window.width/2 - Window.width/4
        self.go_left(instance, None)
        self.schedule_events()
        
    def go_left(self, instance, value):
        '''Move the fleet towards the left edge of the screen. 
        Normally the fleet moves from the right side of the screen.
        If if `value` is None, however, we are calling the animation for the first time,
        in which case, the animation starts from the middle of the screen, so we half the
        nimation duration'''
        if value is None:
            animation = Animation(x=0, d=self.move_delay / 2.0)
        else:
            animation = Animation(x=0, d=self.move_delay)
        animation.bind(on_complete = self.go_right)
        animation.start(self)
    
    def go_right(self, instance, value):
        '''Move the fleet towards the right edge of the screen.'''
        animation = Animation(right=self.parent.width, d=self.move_delay)
        animation.bind(on_complete=self.go_left)
        animation.start(self)
    
    def schedule_events(self):
        '''Start all random events:
        * After a random interval, drop a bomb'''
        Clock.schedule_once(self.bomb, random()) 
        
    def bomb(self, dt):
        '''Randomly choose one of the attackers to drop a bomb, then randomly reschedule'''
        if len(self.survivors):
            child = choice(self.survivors)
            child.invader.drop_bomb()
            Clock.schedule_once(self.bomb, random())

Overwriting fleet.py


In [184]:
%%file invader.py

from kivy.uix.image import Image
from kivy.app import App
from projectile import Bomb

class Invader(Image):
    def drop_bomb(self):
        bomb = Bomb()
        bomb.center = (self.center_x, self.y)
        fleet = self.parent.parent
        fleet.mainscreen.add_widget(bomb)
        bomb.shoot(self.center_x, 0, fleet.mainscreen.player)
        

Overwriting invader.py


In [185]:
%%html
<img src="assets/screenshots/galaxyinvaders-11.png" width=400 />

In [186]:
#!python main.py

### 7.4 Missile and Bomb Sounds

Let's add sound to the missiles and bombs. Obviously, they should sound different. We used *cfxr* (a mac port of *sfxr*) to create some quick-and-dirty sound effects.

In [187]:
%%file projectile.py
from kivy.animation import Animation
from kivy.uix.image import Image
from kivy.logger import Logger
from kivy.app import App
from boom import Boom

class Projectile(Image):
    '''Lauch this piece of ammunition towards a target object (`target`),
    located at coordinates (`tx`, `ty`)'''
    def shoot(self, tx, ty, target):
        self.target = target
        self.animation = Animation(x=tx, y=ty)
        self.animation.bind(on_start=self.on_start)
        self.animation.bind(on_progress=self.on_progress)
        self.animation.bind(on_complete=self.on_stop)
        self.animation.start(self)
        
    def on_start(self, instance, value):
        pass
    
    def on_progress(self, instance, value, progression):
        pass
            
    def on_stop(self, instance, value):
        self.parent.remove_widget(self)
        
class Missile(Projectile):
    def on_start(self, instance, value):
        super(Missile, self).on_start(instance, value)
        App.get_running_app().sounds.play_shoot()
        
class Bomb(Projectile):
    def on_start(self, instance, value):
        super(Bomb, self).on_start(instance, value)
        App.get_running_app().sounds.play_bomb()


Overwriting projectile.py


The result is satisfying. We no longer use the `Boom` effect, however. We will save that for when we add collision detection.

In [188]:
#!python main.py


## 8.0 Collision Detection

We will now make an important change to our projectile class. We will test for **collisions**.
To do this, we will overload the `on_progress` event for our projectile animation to check for a target collision. Notice when we shoot, we give a target object, either the player, or the fleet, as that is the only object that needs to check for collision.

We will add a collision check by calling the target's collision detection routine (`collide_projectile`) in the projectile's `on_progress` event.

In [189]:
%%file projectile.py
from kivy.animation import Animation
from kivy.uix.image import Image
from kivy.logger import Logger
from kivy.app import App
from boom import Boom

class Projectile(Image):
    '''Lauch this piece of ammunition towards a target object (`target`),
    located at coordinates (`tx`, `ty`)'''
    def shoot(self, tx, ty, target):
        self.target = target
        self.animation = Animation(x=tx, y=ty)
        self.animation.bind(on_start=self.on_start)
        self.animation.bind(on_progress=self.on_progress)
        self.animation.bind(on_complete=self.on_stop)
        self.animation.start(self)
        
    def on_start(self, instance, value):
        pass
    
    def on_progress(self, instance, value, progression):
        if self.target.collide_projectile(self):
            self.animation.stop(self)
            
    def on_stop(self, instance, value):
        self.parent.remove_widget(self)
        
class Missile(Projectile):
    def on_start(self, instance, value):
        super(Missile, self).on_start(instance, value)
        App.get_running_app().sounds.play_shoot()
        
class Bomb(Projectile):
    def on_start(self, instance, value):
        super(Bomb, self).on_start(instance, value)
        App.get_running_app().sounds.play_bomb()

Overwriting projectile.py


In [190]:
%%file player.py
from kivy.uix.image import Image
from kivy.logger import Logger
from kivy.properties import NumericProperty
from kivy.graphics import Line, Ellipse
from projectile import Missile

from boom import Boom

class Player(Image):
    lives = NumericProperty(1)
    
    def on_touch_down(self, touch):
        if self.collide_point(*touch.pos):
            self.center_x = touch.x
            touch.grab(self)
            return True
        elif self.player_area.collide_point(*touch.pos):
            self.shoot()
            return True
            
    def on_touch_move(self, touch):
        if touch.grab_current is self:
            self.center_x = touch.x
            return True
        
    def shoot(self):
        '''Shoot straight up.'''
        missile = Missile()
        missile.center = (self.center_x, self.top)
        self.mainscreen.add_widget(missile)
        (fx, fy) = self.center_x, self.mainscreen.height
        missile.shoot(fx, fy, self.mainscreen.fleet)
        
    def collide_projectile(self, projectile):
        '''Detect collision with a bomb'''
        if self.lives and self.collide_widget(projectile):
            boom = Boom()
            boom.pos = self.pos
            self.parent.add_widget(boom)
            self.color = (0,0,0,0)
            self.lives -= 1
            # Check for end of game
            return True
        return False
        

Overwriting player.py


In [191]:
%%file fleet.py
from kivy.uix.gridlayout import GridLayout
from kivy.animation import Animation
from kivy.clock import Clock
from kivy.core.window import Window
from kivy.properties import NumericProperty, ListProperty
from kivy.logger import Logger

from random import choice, random
from dock import Dock
from boom import Boom

class Fleet(GridLayout):
    survivors = ListProperty([])
    move_delay = NumericProperty(3)
    
    def __init__(self, **kwargs):
        super(Fleet, self).__init__(**kwargs)
        for x in range (0,32):
            dock = Dock()
            self.survivors.append(dock)
            self.add_widget(dock)
    
    def start_fleet(self, instance=None, value=None):
        '''Start the fleet march'''
        self.x = Window.width/2 - Window.width/4
        self.go_left(instance, None)
        self.schedule_events()
        
    def go_left(self, instance, value):
        '''Move the fleet towards the left edge of the screen. 
        Normally the fleet moves from the right side of the screen.
        If if `value` is None, however, we are calling the animation for the first time,
        in which case, the animation starts from the middle of the screen, so we half the
        nimation duration'''
        if value is None:
            animation = Animation(x=0, d=self.move_delay / 2.0)
        else:
            animation = Animation(x=0, d=self.move_delay)
        animation.bind(on_complete = self.go_right)
        animation.start(self)
    
    def go_right(self, instance, value):
        '''Move the fleet towards the right edge of the screen.'''
        animation = Animation(right=self.parent.width, d=self.move_delay)
        animation.bind(on_complete=self.go_left)
        animation.start(self)
    
    def schedule_events(self):
        '''Start all random events:
        * After a random interval, drop a bomb'''
        Clock.schedule_once(self.bomb, random()) 
        
    def bomb(self, dt):
        '''Randomly choose one of the attackers to drop a bomb, then randomly reschedule'''
        if len(self.survivors):
            child = choice(self.survivors)
            child.invader.drop_bomb()
            Clock.schedule_once(self.bomb, random())
            
    def collide_projectile(self, projectile):
        '''Detect a collision with projectile. Loop through remaining children checking for collision'''
        for child in self.survivors:
            if child.invader.collide_widget(projectile):
                boom = Boom()
                boom.pos = child.pos
                child.canvas.clear()
                child.add_widget(boom)
                self.survivors.remove(child)
                return True
        return False


Overwriting fleet.py


In [192]:
%%html
<img src="assets/screenshots/galaxyinvaders-13.png" width=400 />

In [193]:
#!python main.py

### 8.1 Game Over
It's starting to feel like an arcade game. Now we need to handle the case when the player dies. There are two things here
1. If the player is out of lives, he shouldn't be able to shoot or move
2. When the game is over, we need to display a 'Game Over' banner

We can handle lives using a kivy property and its associated `on_` handler. The ship is invisible and immobile if lives == 0


How do we keep a player from shooting or moving? For now, we will add an `active` flag, and ignore all events if this attribute is set to `False`.

In [194]:
%%file player.py
from kivy.uix.image import Image
from kivy.logger import Logger
from kivy.properties import NumericProperty
from kivy.graphics import Line, Ellipse
from projectile import Missile

from boom import Boom

class Player(Image):
    lives = NumericProperty(0)
    active = False
    
    def add_life(self, instance, value):
        self.lives += 1
        
    def on_touch_down(self, touch):
        if not self.active:
            return False
        if self.collide_point(*touch.pos):
            self.center_x = touch.x
            touch.grab(self)
            return True
        elif self.player_area.collide_point(*touch.pos):
            self.shoot()
            return True
            
    def on_touch_move(self, touch):
        if not self.active:
            return False
        if touch.grab_current is self:
            self.center_x = touch.x
            return True
        
    def shoot(self):
        '''Shoot straight up.'''
        if not self.active:
            return
        missile = Missile()
        missile.center = (self.center_x, self.top)
        self.mainscreen.add_widget(missile)
        (fx, fy) = self.center_x, self.mainscreen.height
        missile.shoot(fx, fy, self.mainscreen.fleet)
        
    def collide_projectile(self, projectile):
        '''Detect collision with a bomb'''
        if self.active and self.lives and self.collide_widget(projectile):
            boom = Boom()
            boom.pos = self.pos
            self.parent.add_widget(boom)
            self.lives -= 1
            return True
        return False
    
    def on_lives(self, instance, value):
        '''If lives drops to zero, the player becomes immobile, invisible, and the game ends.'''
        if self.lives == 0:
            self.active = False
            self.color = (0,0,0,0)
            self.mainscreen.game_over()
        else:
            self.active = True
            self.color = (1,1,1,1)
        

Overwriting player.py


In main.py, add a 'Game Over' message. Then check that a player is active before allow key commands. Finally, increase a player's lives to start the game.

In [195]:
%%file main.py
from kivy.app import App
from kivy.uix.floatlayout import FloatLayout
from kivy.uix.gridlayout import GridLayout
from kivy.lang.builder import Builder
from kivy.core.window import Window
from kivy.logger import Logger
from kivy.core.audio import SoundLoader
from kivy.clock import Clock
from kivy.uix.label import Label
from kivy.animation import Animation

from player import Player
from sounds import Sounds
from fleet import Fleet

Builder.load_file('sprites.kv')


class GalaxyInvaders(FloatLayout):
    playing = False
    unlocked = False
    message = None

    def __init__(self, **kwargs):
        super(GalaxyInvaders, self).__init__(**kwargs)
        self._keyboard = Window.request_keyboard(self.close, self)
        self._keyboard.bind(on_key_down=self.press)
        label = Label(text="Press [b]S[/b] to start\n[b]<- ->[/b] to move\n[b]Space[/b] to shoot\n[b]Esc[/b] to quit",
                     halign='center', markup=True)
        self.add_widget(label)
        self.help_text = label
        
    def start_game(self):
        '''Display a  'ready' message before a game starts.
        When the animation is done, start the game.'''
        label = Label(text='Ready!')
        animation = Animation(font_size=72, d=2)
        animation += Animation(font_size=0, d=2)
        self.add_widget(label)
        self.message = label
        animation.bind(on_complete=self.fleet.start_fleet)
        animation.bind(on_complete=self.remove_message)
        animation.bind(on_complete=self.player.add_life)
        animation.start(label)
        
    def remove_message(self, instance, value):
        '''Remove the message text when the animation is done'''
        self.remove_widget(self.message)
        self.message = None

    def close(self):
        self._keyboard.unbind(on_key_down=self.press)
        self._keyboard = None
        # Eventually, we should do an 'Are You Sure?' prompt. For now, just quit
        App.get_running_app().stop() 
        
    def press(self, keyboard, keycode, text, modifiers):
        '''Handle key commands'''
        if keycode[1] == 'escape':
                self.close()
                return True

        if not self.playing:
            if keycode[1] == 's':
                self.remove_widget(self.help_text)
                self.playing = True
                self.start_game()
        elif self.player.active:
            if keycode[1] == 'left':
                self.player.x -= 30
                if self.player.x < self.x:
                    self.player.x = self.x
            elif keycode[1] == 'right':
                self.player.x += 30
                if self.player.x > self.width - self.player.width:
                    self.player.x = self.width - self.player.width
            elif keycode[1] == 'spacebar':
                self.player.shoot()
            else:
                Logger.debug("Unknown key: {}".format(keycode))
        return True
    
    def game_over(self):
        label = Label(text='Game Over!')
        animation = Animation(font_size=72, d=2)
        animation += Animation(font_size=0, d=1)
        self.add_widget(label)
        self.message = label
        animation.bind(on_complete=self.remove_message)
        animation.start(label)


class GalaxyInvadersApp(App):
    sounds = Sounds()
    def build(self):
        return GalaxyInvaders()

if __name__ == '__main__':
    GalaxyInvadersApp().run()

Overwriting main.py


In [196]:
%%html
<img src='assets/screenshots/galaxyinvaders-12.png' width=400 />

In [197]:
#!python main.py

## Fun with Properties: Modes and Crawls

It would be nice if, when the game was over, everything returned to the demo screen so that the player could play again.

We saw the usefulness of properties (and their associated event handlers) when checking for game over. We can use these properties to add modes for demo and gameplay.

First, we have to refactor our code to start/stop the gameplay. Let's create a kivy property called `mode`, and use that to change between behaviors.


In [198]:
%%file main.py
from kivy.app import App
from kivy.uix.floatlayout import FloatLayout
from kivy.uix.gridlayout import GridLayout
from kivy.lang.builder import Builder
from kivy.core.window import Window
from kivy.logger import Logger
from kivy.core.audio import SoundLoader
from kivy.clock import Clock
from kivy.uix.label import Label
from kivy.animation import Animation
from kivy.properties import StringProperty

from player import Player
from sounds import Sounds
from fleet import Fleet

Builder.load_file('sprites.kv')


class GalaxyInvaders(FloatLayout):
    mode = StringProperty('init')
    message = None

    def __init__(self, **kwargs):
        super(GalaxyInvaders, self).__init__(**kwargs)
        self._keyboard = Window.request_keyboard(self.close, self)
        self._keyboard.bind(on_key_down=self.press)
        self.demo_mode()

    def on_mode(self, instance, value):
        if value == 'demo':
            self.fleet.stop_fleet(instance, value)
            self.player.lives = 0


            label = Label(text="Press [b]S[/b] to start\n"
                          "[b]<- ->[/b] to move\n"
                          "[b]Space[/b] to shoot\n"
                          "[b]Esc[/b] to quit",
                         halign='center', markup=True)
            self.add_widget(label)
            self.help_text = label
            self.player.x = Window.width / 2
            self.fleet.fill_fleet()
        elif value == 'game':
            self.remove_widget(self.help_text)
            self.player.add_life(instance, value)
            self.player.active = True
        
    def demo_mode(self, instance=None, value=None):
        self.mode = 'demo'
        
    def game_mode(self, instance, value):
        self.mode = 'game'
        
    def start_game(self):
        label = Label(text='Ready!')
        animation = Animation(font_size=72, d=2)
        animation += Animation(font_size=0, d=2)
        self.add_widget(label)
        self.message = label
        animation.bind(on_complete=self.fleet.start_fleet)
        animation.bind(on_complete=self.remove_message)
        animation.bind(on_complete=self.game_mode)
        animation.start(label)
        
    def remove_message(self, instance, value):
        '''Remove the message text when the animation is done'''
        self.remove_widget(self.message)
        self.message = None

    def close(self):
        self._keyboard.unbind(on_key_down=self.press)
        self._keyboard = None
        # Eventually, we should do an 'Are You Sure?' prompt. For now, just quit
        App.get_running_app().stop() 
        
    def press(self, keyboard, keycode, text, modifiers):
        '''Handle key commands, different, depending on game mode'''
        if keycode[1] == 'escape':
                self.close()
                return True

        if self.mode == 'demo':
            if keycode[1] == 's':
                self.remove_widget(self.help_text)
                self.start_game()
                self.mode = 'loading' # to prevent multiple starts
        elif self.mode == 'game' and self.player.active:
            if keycode[1] == 'left':
                self.player.x -= 30
                if self.player.x < self.x:
                    self.player.x = self.x
            elif keycode[1] == 'right':
                self.player.x += 30
                if self.player.x > self.width - self.player.width:
                    self.player.x = self.width - self.player.width
            elif keycode[1] == 'spacebar':
                self.player.shoot()
            else:
                Logger.debug("Unknown key: {}".format(keycode))
        return True
    
    def game_over(self, message='Game Over!'):
        '''Animate a 'Game Over' message (save it as an attribute, `self.message` so we can delete it later).
        '''
        label = Label(text=message)
        animation = Animation(font_size=72, d=2)
        animation += Animation(font_size=0, d=1)
        self.add_widget(label)
        self.message = label
        animation.bind(on_complete=self.remove_message)
        animation.bind(on_complete=self.demo_mode)
        animation.start(label)
            

class GalaxyInvadersApp(App):
    sounds = Sounds()
    def build(self):
        return GalaxyInvaders()

if __name__ == '__main__':
    GalaxyInvadersApp().run()

Overwriting main.py


### Replay
There are a couple of things left to do to make toggling between game and demo mode work. If we want to be able to play again after a game ends, we will need to reset the fleet and player to their starting states after a game is over. This means:
*  The fleet gets a a full set of invaders (and moves back to the center of the screen), and
* The player gets a full set of lives


We will start by refilling empty docks with invaders. We can do this by refactoring the fill we do at init time.
To handle this refilling, we will have to keep track of docks, and not just survivors

In [199]:
%%file fleet.py
from kivy.uix.gridlayout import GridLayout
from kivy.animation import Animation
from kivy.clock import Clock
from kivy.core.window import Window
from kivy.properties import NumericProperty, ListProperty, BooleanProperty
from kivy.logger import Logger

from random import choice, random
from dock import Dock
from boom import Boom


class Fleet(GridLayout):
    survivors = ListProperty([])
    move_delay = NumericProperty(3)
    crawl = BooleanProperty(False)
    docks = []
    
    def __init__(self, **kwargs):
        super(Fleet, self).__init__(**kwargs)
        self.fill_fleet()
        
    def fill_fleet(self):
        '''Fill up missing spots in the dock. This can only be done if the crawl is off'''
        if self.crawl:
            return
        for dock in self.docks:
            self.remove_widget(dock)
        self.survivors = []    
        for x in range(32):
            dock = Dock()
            self.survivors.append(dock)
            self.add_widget(dock)
            self.docks.append(dock)
            
    def start_fleet(self, instance=None, value=None):
        '''Start the fleet march'''
        self.x = Window.width/2 - Window.width/4
        self.crawl = True

    def on_crawl(self, instance, value):
        '''Transition from doing the fleet crawl and firing (True) 
        to sitting motionless in the center (False)'''
        if value == True:
            self.go_left(instance, None)
            self.schedule_events()
        else:
            Animation.cancel_all(self)
            # back to center
            animation = Animation(x=Window.width/2 - Window.width/4)
            animation.start(self)

    def stop_fleet(self, instance=None, value=None):
        '''stop the fleet march'''
        self.crawl = False
        
    def go_left(self, instance, value):
        '''Move the fleet towards the left edge of the screen. 
        Normally the fleet moves from the right side of the screen.
        If if `value` is None, however, we are calling the animation for the first time,
        in which case, the animation starts from the middle of the screen, so we half the
        animation duration'''
        if value is None:
            animation = Animation(x=0, d=self.move_delay / 2.0)
        else:
            animation = Animation(x=0, d=self.move_delay)
        animation.bind(on_complete = self.go_right)
        animation.start(self)
    
    def go_right(self, instance, value):
        '''Move the fleet towards the right edge of the screen.'''
        animation = Animation(right=self.parent.width, d=self.move_delay)
        animation.bind(on_complete=self.go_left)
        animation.start(self)
    
    def schedule_events(self):
        '''Start all random events:
        * After a random interval, drop a bomb'''
        Clock.schedule_once(self.bomb, random()) 
        
    def bomb(self, dt):
        '''Randomly choose one of the attackers to drop a bomb, then randomly reschedule'''
        if len(self.survivors):
            child = choice(self.survivors)
            child.invader.drop_bomb()
            if self.crawl:
                Clock.schedule_once(self.bomb, random())
            
    def collide_projectile(self, projectile):
        '''Detect a collision with projectile. Loop through remaining children checking for collision'''
        for child in self.survivors:
            if child.invader.collide_widget(projectile):
                boom = Boom()
                boom.pos = child.pos
                child.canvas.clear()
                child.add_widget(boom)
                self.survivors.remove(child)
                return True
        return False

Overwriting fleet.py


In [200]:
#!python main.py

Finally, let's get rid of the debug frames and see what it looks like!


In [201]:
%%file galaxyinvaders.kv
##:include debug.kv
<GalaxyInvaders>:
    id: _mainscreen
    enemy_area: _enemy_area
    player_area: _player_area
    player: _player
    fleet: _fleet
    BoxLayout:
        orientation: 'vertical'
        FloatLayout:
            id: _enemy_area
            size_hint: 1, 0.7
            Fleet:
                id: _fleet
                mainscreen: _mainscreen
                size_hint: .5, .4
                pos_hint: {'top': .9}
                x: root.width/2 - root.width/4
                cols: 8
                spacing: 20
                    
        FloatLayout:
            id: _player_area
            size_hint: 1, 0.3
            Player:
                id: _player
                mainscreen: _mainscreen
                player_area: _player_area
                x: self.parent.width / 2

Overwriting galaxyinvaders.kv


In [204]:
%%html
<img src="assets/screenshots/galaxyinvaders-14.png" width=400 />


In [203]:
#!python main.py

## What's next?
There's lots more we can do with this came. Some features you may want to add:
* Score
* Additional levels (when all invaders are killed)
* increasing difficulty with levels
* Multiple player lives
* solo attacks for invaders
* Smooth player movement
* random gameplay in demo mode


Have fun!